# Firefighting cost prediction with boosted trees
By Gabriel C. Ullmann, 2023

In this tutorial, we show you the steps to train a boosted tree classification model, and also how to evaluate this model afterwards.

## Import libraries
First of all, we import xgboost, the library which provides boosted tree functionality. The purpose other libraries will be explained as we advance in the tutorial.

In [7]:
import xgboost
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

## Load data
We use Pandas to load the preprocessed version of our dataset (london_clean.csv) and select only the columns we will use for training the tree. We are aware the unecessary columns could have been removed in the preprocessing phase, but we chose to keep them in the file so we can experiment with different combination of inputs/outputs.

In [3]:
ds = pd.read_csv('preprocessing/data/london_clean.csv')
selected_cols = ["DateOfCall", "PropertyType", "NumPumpsAttending", 
                    "PumpHoursRoundUp", "CostCat", "mean_temp"]
ds = ds.loc[:, selected_cols]

## Split data
After loading the data, we start by splitting the dataset between columns that will be used as input to train the model (X) and the actual output (y). In other words, for a certain combination of data about the file (e.g. data of call, property type, etc.) we want to predict its cost category.

Besides splitting columns, we also want to split the rows to create two datasets: one for training (70% of the original dataset rows) and testing (30% of the original dataset rows). We use train_test_split from scikit-learn to do this task for us.

In [4]:
X = ds.drop(["CostCat"], axis=1).values 
y = ds["CostCat"].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.7, test_size=0.3, random_state=42
)

## Train model
We call xgboost.XGBClassifier and use the training dataset (x_train, y_train) to train the model.

In [5]:
model = xgboost.XGBClassifier(objective="binary:logistic", random_state=42)
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

## Make predictions
We use the trained model to make predictions. We then compare these predictions to the actual labels to check how accurate is the model. According to sci-kit learn's classification_report, the accuracy (average f1-score) is 0.77.

In [6]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.74      0.90      0.82    210199
           1       0.76      0.48      0.59    127674
           2       1.00      1.00      1.00     31404
           3       0.70      0.91      0.79      4963
           4       0.72      0.42      0.53      4546
           5       0.91      0.96      0.93      7200

    accuracy                           0.77    385986
   macro avg       0.80      0.78      0.78    385986
weighted avg       0.77      0.77      0.75    385986

